In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
import re
import pandas as pd
import numpy as np

def extract_flight_data(origins, destinations, startdates):
    flight_data = {
        "Origin": [],
        "Destination": [],
        "StartDate": [],
        "DepartureTime": [],
        "ArrivalTime": [],
        "Price": [],
        "Airline": [],
        "Stops": [],
        "LayoverAirports": [],
        "TravelTime": [],
        "ClassType": []
    }

    # Set Chrome options and the user-agent
    opts = Options()
    opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

    # Initialize the ChromeDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

    for origin in origins:
        for destination in destinations:
            for startdate in startdates:
                url = f"https://www.kayak.cl/flights/{origin}-{destination}/{startdate}?sort=bestflight_a"
                # Open the Kayak website
                driver.get(url)
                
                sleep(5)
                soup = BeautifulSoup(driver.page_source, 'lxml')

                if soup.find_all('p')[0].getText() == "Please confirm that you are a real KAYAK user.":
                    print("Kayak thinks I'm a bot, which I am ... so let's wait a bit and try again")
                    driver.close()
                    sleep(20)
                    continue 


                # Presionar multiples veces 'ver mas resultaods' para obtener la mayor cantidad de vuelos posibles por búsqueda
                while True:
                    try:
                        boton = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//div[@role="button" and @class="ULvh-button show-more-button"]')))
                        boton.click()
                    except:
                        print("Button disappeared or not clickable. Stopping.")
                        break

                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Extraer Horarios de salida y llegada
                deptime = []
                arrtime = []
                div_elements_times = soup.find_all('div', class_='vmXl vmXl-mod-variant-large')

                for div in div_elements_times:
                    spans = div.find_all('span')
                    if len(spans) == 3:
                        deptime.append(spans[0].get_text())
                        arrtime.append(spans[2].get_text())

                # Extraer todos los precios
                prices_int = []
                div_elements_price = soup.find_all('div', class_='f8F1-price-text')

                for div in div_elements_price:
                    price_text = div.get_text()
                    price_text = price_text.replace('$', '').replace('.', '')
                    prices_int.append(int(price_text))

                # Extraer lineas aereas
                lineas = []
                div_elements_lineas = soup.find_all('div', class_='J0g6-operator-text')

                for div in div_elements_lineas:
                    lineas_text = div.get_text()
                    lineas.append(lineas_text)

                # Extraer lineas aereas
                escalas = []
                span_elements_escalas = soup.find_all('span', class_='JWEO-stops-text')

                for div in span_elements_escalas:
                    escalas_text = div.get_text()
                    escalas.append(escalas_text)

                # Extraer aeropuertos de escalas
                cleaned_airport_codes = []
                div_elements_jweo = soup.find_all('div', class_='JWEO')

                for div_jweo in div_elements_jweo:
                    div_airport = div_jweo.find('div', class_='c_cgF c_cgF-mod-variant-full-airport')
                    span_elements = div_airport.find_all('span')
                    airport_codes = [span.get_text() for span in span_elements]
                    cleaned_airport_code = ', '.join(set(filter(None, airport_codes)))
                    cleaned_airport_codes.append(cleaned_airport_code)

                cleaned_airport_codes = [', '.join(set(code.split(', '))) for code in cleaned_airport_codes]

                # Extraer horas del viaje
                horas_viaje = []
                div_elements_horas = soup.find_all('div', class_='xdW8 xdW8-mod-full-airport')

                for div in div_elements_horas:
                    horas_text = div.find('div', class_='vmXl vmXl-mod-variant-default').get_text()
                    horas_viaje.append(horas_text)

                # Extraer tipo de clase
                tipo_clase = []
                div_M_JD = soup.find_all('div', class_='M_JD')

                for div in div_M_JD:
                    div_aC3z_links = div.find('div', class_='aC3z-links')
                    div_aC3z_option = div_aC3z_links.find('div', class_='aC3z-option')
                    div_aC3z_name = div_aC3z_option.find('div', class_='aC3z-name').get_text()
                    tipo_clase.append(div_aC3z_name)

                # Agregar los datos a la estructura de datos
                for i in range(len(deptime)):
                    flight_data["Origin"].append(origin)
                    flight_data["Destination"].append(destination)
                    flight_data["StartDate"].append(startdate)
                    flight_data["DepartureTime"].append(deptime[i])
                    flight_data["ArrivalTime"].append(arrtime[i])
                    flight_data["Price"].append(prices_int[i])
                    flight_data["Airline"].append(lineas[i])
                    flight_data["Stops"].append(escalas[i])
                    flight_data["LayoverAirports"].append(cleaned_airport_codes[i])
                    flight_data["TravelTime"].append(horas_viaje[i])
                    flight_data["ClassType"].append(tipo_clase[i])

    # Cerrar el navegador al finalizar
    driver.quit()

    return flight_data

# Definir las listas de origen, destino y fechas
origins = ['SCL']
destinations = ['PAR']
startdates = ["2023-12-11", "2023-12-12", "2023-12-13","2023-12-20","2023-12-22","2023-12-24","2023-12-25"]

# Llamar a la función para extraer datos
flight_data = extract_flight_data(origins, destinations, startdates)

# Convertir los datos a un DataFrame de Pandas
df = pd.DataFrame(flight_data)



FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [4]:
df

,Origin,Destination,StartDate,DepartureTime,ArrivalTime,Price,Airline,Stops,LayoverAirports,TravelTime,ClassType
0,SCL,ATL,2023-12-11,5:10,10:15+1,257997,"Copa Airlines, Frontier",2 escalas,", PTY, MIA",31h 05m,Turista
1,SCL,ATL,2023-12-11,23:15,13:37+1,346180,"LATAM Airlines, Frontier",1 escala,JFK-LGA,16h 22m,Basic
2,SCL,ATL,2023-12-11,5:10,10:15+1,257997,"Copa Airlines, Frontier",2 escalas,", PTY, MIA",31h 05m,Turista
3,SCL,ATL,2023-12-11,20:55,10:15+1,315902,"LATAM Airlines, Frontier",2 escalas,", LIM, MIA",15h 20m,Light
4,SCL,ATL,2023-12-11,23:20,6:55+1,891353,LATAM Airlines,directo,,9h 35m,Light
...,...,...,...,...,...,...,...,...,...,...,...
1576,SCL,MIA,2023-12-25,5:07,16:45,764683,"JetSMART, Avianca",1 escala,BOG,13h 38m,Economy XS
1577,SCL,MIA,2023-12-25,20:55,10:59+1,961753,LATAM Airlines,2 escalas,", LIM, BOG",16h 04m,Light
1578,SCL,MIA,2023-12-25,8:50,21:29,864659,"LATAM Airlines, Copa Airlines",2 escalas,", LIM, PTY",14h 39m,Basic
1579,SCL,MIA,2023-12-25,16:40,6:20+1,1043759,Delta,1 escala,LIM,15h 40m,Cabina principal


In [5]:
import sqlite3



# Conectar a la base de datos SQLite
conn = sqlite3.connect('master.db')

# Guardar el DataFrame en una tabla llamada 'personas'
df.to_sql('vuelos', conn, if_exists='replace', index=False)

# Cerrar la conexión
conn.close()
